### Knowledge and Data Mining Demo

## Minesweeper 

Leonardo Amato (ID:2028621), Lorenzo Corrado (ID:2020623)

October 05, 2022

# Libraries

In [3]:
!pip3 install python-sat
from pysat.solvers import Minisat22
import itertools
import random
import queue


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


# Implementation

##Utils
utils used by the various methods



In [4]:
def findsubsets(s, n):
    return [list(i) for i in itertools.combinations(s, n)]

def compute_all_position():
  position_list = []
  for x in range(GRID_HEIGHT):
    for y in range(GRID_WIDTH):
      position_list.append((x,y))
  return position_list

def compute_adj_position(x,y):
  all_adj_position = {(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)}
  filtered_adj_position = []
  for (x_off,y_off) in all_adj_position:
    new_x = x+x_off
    new_y = y+y_off
    if(new_x >= 0 and new_y >= 0 and new_x < GRID_HEIGHT and new_y < GRID_WIDTH):
      filtered_adj_position.append((new_x,new_y))
  return filtered_adj_position

def check_number_in_position(game_state, x, y, min, max):
  for n in range (min,max):
    if ((n,x,y) in game_state):
      return n
  return None

In [5]:
def game_state_print(game_state,mine_set,lost_position,won):
    string = "" 
    string += "╔" 
    for x in range(GRID_WIDTH):
      string += "═══" 
      if(x < GRID_WIDTH - 1):
        string += "╦" 
    string += "╗" 
    string += "\n"
    for x in range(GRID_HEIGHT):
      for y in range(GRID_WIDTH):
        if (x,y) == lost_position:
          string += "║X☼X" 
        else: 
          if((x,y) in mine_set):
            if won:
              string += "║▓☼▓" 
            else:
              string += "║▓▓▓" 
          else: 
            number = check_number_in_position(game_state,x,y,1,8)
            if(number is not None):
                string += "║ " + str(number) + " "
            else:
              if (0,x,y) in game_state:
                string += "║   " 
              else:
                string += "║░░░" 
      string += "║\n"
      if(x < GRID_HEIGHT - 1):
        string += "╠" 
        for x in range(GRID_WIDTH):
          string += "═══" 
          if(x < GRID_WIDTH - 1):
            string += "╬" 
        string += "╣" 
        string += "\n"
    
    string += "╚" 
    for x in range(GRID_WIDTH):
      string += "═══" 
      if(x < GRID_WIDTH - 1):
        string += "╩" 
    string += "╝" 
    string += "\n"
    print(string)

##Game state to CNF conversion
Methods used to convert the state of the game to a CNF formula

In [6]:
def Mine(r,c):
    return r*GRID_WIDTH + c + 1

In [7]:
def number_of_mines_rule(minesweaper, number_set, number_of_mines):
    ##RULE: there are n mine in the field
    if(number_of_mines > -1):
      all_positions = compute_all_position()
      for position in all_positions:
        if check_number_in_position(number_set,position[0],position[1],0,8) is not None:
          all_positions.remove(position)
      for adj_subset in findsubsets(all_positions, (GRID_WIDTH*GRID_HEIGHT)-number_of_mines+1):
        mine_position_list = []
        for (x_off,y_off) in adj_subset:
          mine_position_list.append((x_off,y_off))
        if len(mine_position_list) > 0:
          minesweaper.add_clause([Mine(i,j) for (i,j) in mine_position_list] )
      for adj_subset in findsubsets(all_positions, number_of_mines+1):
        mine_position_list = []
        for (x_off,y_off) in adj_subset:
          mine_position_list.append((x_off,y_off))
        if len(mine_position_list) > 0:
          minesweaper.add_clause([-Mine(i,j) for (i,j) in mine_position_list])

In [8]:
def state_to_CNF(minesweaper, numbers_set):
  #converting number to cnf knowledge
  #for each number
  for (number,x,y) in numbers_set:
    #if we have a number the position is safe
    minesweaper.add_clause([-Mine(x,y)])
    if(number == 0):
      continue
    #we get the adj positions
    adj_position = compute_adj_position(x,y)
    
    #we have at least N mines in the adj positions
    for adj_subset in findsubsets(adj_position, len(adj_position)-number+1):
      mine_position_list = []
      for (x_off,y_off) in adj_subset:
        mine_position_list.append((x_off,y_off))
      if len(mine_position_list) > 0:
        minesweaper.add_clause([Mine(i,j) for (i,j) in mine_position_list] )

    #we have at most N mines in the adj positions
    for adj_subset in findsubsets(adj_position, number+1):
      mine_position_list = []
      for (x_off,y_off) in adj_subset:
        mine_position_list.append((x_off,y_off))
      if len(mine_position_list) > 0:
          minesweaper.add_clause([-Mine(i,j) for (i,j) in mine_position_list])
      



## Field section
This section is used to encapsulate the field to make so that the player can't directly see it

In [9]:
##Field
def GenerateMineSweeperMap(height, width, number_of_mines):
    field = [[0 for column in range(width)] for row in range(height)]
    for num in range(number_of_mines):
        while (True):
          i = random.randint(0,height-1)
          j = random.randint(0,width-1)
          if (field[i][j] != 'X'):
            break
        field[i][j] = 'X'
        for adj_pos in compute_adj_position(i,j):
          if field[adj_pos[0]][adj_pos[1]] != 'X':
              field[adj_pos[0]][adj_pos[1]] += 1
    return field

def box_check(grid,i,j,first_round):
    if(grid[i][j] == "X" and first_round):
      grid[i][j] = 0
      for adj_pos in compute_adj_position(i,j):
          if grid[adj_pos[0]][adj_pos[1]] != 'X':
              grid[adj_pos[0]][adj_pos[1]] -= 1
          else:
              grid[i][j] += 1
      while (True):
          x = random.randint(0,GRID_HEIGHT-1)
          y = random.randint(0,GRID_WIDTH-1)
          if (grid[x][y] != 'X' and (y,x) != (i,j)):
            break
      grid[x][y] = 'X'
      for adj_pos in compute_adj_position(x,y):
        if grid[adj_pos[0]][adj_pos[1]] != 'X':
          grid[adj_pos[0]][adj_pos[1]] += 1
        

    zero_box = queue.Queue()
    checked_box = set()
    output = set()
    if(grid[i][j] != 'X'):
      if(grid[i][j] != 0):
        output.add((grid[i][j],i,j))
      if(grid[i][j] == 0):
        zero_box.put((i,j))
        while(not zero_box.empty()):
          position = zero_box.get()
          output.add((0,position[0],position[1]))
          checked_box.add(position)
          for adj_pos in compute_adj_position(position[0],position[1]):
            if(adj_pos not in checked_box):
              if(grid[adj_pos[0]][adj_pos[1]] == 0):
                zero_box.put(adj_pos)
              else:
                output.add((grid[adj_pos[0]][adj_pos[1]],adj_pos[0],adj_pos[1]))
                checked_box.add(adj_pos)
    return output

def winning_check(mine_set, grid):
  count = 0
  for position in mine_set:
    if(grid[position[0]][position[1]] == 'X'):
      count += 1
    else:
      return False
  return count == N_MINES


        


## Player decision section
In the following section we have all the function used by the player to infer new knowledge about the field

In [10]:
##Player
def infer_new_knowledge(number_set, number_of_mines):#we check which position are safe, which position have a mine, and which are unknown

  minesweaper = Minisat22()

  number_of_mines_rule(minesweaper,number_set,number_of_mines)
  state_to_CNF(minesweaper, number_set)
  set_of_safe = set()
  set_of_mine = set()
  assumptions = []
  suggest = False

  for x in range(GRID_HEIGHT):
    for y in range(GRID_WIDTH):
      check_mine = minesweaper.solve(assumptions=[Mine(x,y)])
      check_safe = minesweaper.solve(assumptions=[-Mine(x,y)])
      if(check_mine and not check_safe):
        set_of_mine.add((x,y))
        assumptions.append(Mine(x,y))
      if(check_safe and not check_mine):
        assumptions.append(-Mine(x,y))
        number = check_number_in_position(number_set,x,y,0,8)
        if(number is None):
          set_of_safe.add((x,y))
  if(len(set_of_safe) == 0):
    en = minesweaper.enum_models()
    prob_matrix = [[0 for _ in range(GRID_WIDTH)] for _ in range(GRID_HEIGHT)]
    for model in en:
      for coordinate in model:
        if(coordinate < 0):
          coordinate = -(coordinate) - 1
          prob_matrix[coordinate // GRID_WIDTH ][coordinate % GRID_WIDTH] += 1

    model_counting_list = []

    for x in range(GRID_HEIGHT):
      for y in range(GRID_WIDTH):
        number = check_number_in_position(number_set,x,y,0,8)
        if(number is None):
          model_counting_list.append((prob_matrix[x][y],x,y))
    random.shuffle(model_counting_list)
    max_value = max(model_counting_list, key = (lambda triplet : triplet[0]))
    set_of_safe.add((max_value[1],max_value[2]))
    suggest = True
  return set_of_safe, set_of_mine, suggest

## Minesweeper parameters

In [11]:
GRID_WIDTH = 3
GRID_HEIGHT = 6
N_MINES = 3
random.seed(1)

In [19]:
GRID_WIDTH = 8
GRID_HEIGHT = 10
N_MINES = 13
random.seed(1)

In [13]:
GRID_WIDTH = 4
GRID_HEIGHT = 4
N_MINES = 5
random.seed(1)

## Game
Given the parameters, this section is going to create a field and let the "player" make the various move to try to uncover all the mines

In [20]:
print(GRID_HEIGHT,"x",GRID_WIDTH,"Field with",N_MINES,"Mines\n")
grid = GenerateMineSweeperMap(GRID_HEIGHT,GRID_WIDTH,N_MINES)
mine_set = set()
number_set = set()
first_turn = True
fail = False
lost_position = None
game_state_print(number_set,mine_set,lost_position,False)
while(True):
  print("The player is thinking \n")
  safe,mine,suggest = infer_new_knowledge(number_set,N_MINES)
  for pos_mine in mine:
    mine_set.add(pos_mine)
  
  if len(mine) == 0:
    print("Player hasn't discovered any new mine")
  else:
    print("Player has discovered that ",mine," have mines")
  if suggest:
    print("Player thinks that ",safe," might be safe")
  else:
    print("Player knows that ",safe," are safe")
  print("\n-----------------------------------------------------------------")

  
  if(not winning_check(mine_set,grid)):
    print("checking all the safe positions and flagging mines")
  for position in safe:
    new_safe = box_check(grid,position[0],position[1],first_turn)
    if(len(new_safe) == 0):
      lost_position = position
      fail = True
      break
    for pos_num in new_safe:
      number_set.add(pos_num)
  if(winning_check(mine_set,grid)):
    break
  if fail:
    break
  game_state_print(number_set,mine_set,lost_position,False)
  first_turn = False

if(fail):
  print("The player has stepped on a mine")
else:
  print("The player has found all the mines")
game_state_print(number_set,mine_set,lost_position,True)


10 x 8 Field with 13 Mines

╔═══╦═══╦═══╦═══╦═══╦═══╦═══╦═══╗
║░░░║░░░║░░░║░░░║░░░║░░░║░░░║░░░║
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║░░░║░░░║░░░║░░░║░░░║░░░║░░░║░░░║
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║░░░║░░░║░░░║░░░║░░░║░░░║░░░║░░░║
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║░░░║░░░║░░░║░░░║░░░║░░░║░░░║░░░║
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║░░░║░░░║░░░║░░░║░░░║░░░║░░░║░░░║
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║░░░║░░░║░░░║░░░║░░░║░░░║░░░║░░░║
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║░░░║░░░║░░░║░░░║░░░║░░░║░░░║░░░║
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║░░░║░░░║░░░║░░░║░░░║░░░║░░░║░░░║
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║░░░║░░░║░░░║░░░║░░░║░░░║░░░║░░░║
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║░░░║░░░║░░░║░░░║░░░║░░░║░░░║░░░║
╚═══╩═══╩═══╩═══╩═══╩═══╩═══╩═══╝

The player is thinking 

